# 特征提取

In [4]:
import sys, os, gc, pickle
import numpy as np
import pandas as pd
import scipy
# import modin.pandas as pd
sys.path.append("./common")
sys.path.append("./dongdong")
sys.path.append("./dongdong/utils")
from np_utils import try_divide
from utils import ReadCSV, Timer, ExtractFeature, ORI_TRAIN_NAMES, ORI_TEST_NAMES, AnalysisCSV, \
FuncMap2, FuncMap1, AnalysisHDF, ExtractFeature1
import dist_utils, ngram_utils
base_feature_save_dir = "/home/kesci/work/common/features/"
tfidf_model_file = "/home/kesci/work/common/util_models/tfidf_word_ngram-13_online_allUnique.bin"
word2vec_kv_model_file = "/home/kesci/work/common/util_models/word2vec_online_concatHorizontal.kv"
word2vec_embed_size = 100
from gensim.models import Word2Vec
# True: 离线训练(100w)
# False: 上线训练(5000w)
OFFLINE = True
train_name = ORI_TRAIN_NAMES
if OFFLINE:
    base_prefix = "debug_"
    train_file = "/home/kesci/work/inputs/train_flast_400w.csv"
    test_file = "/home/kesci/work/inputs/train_flast_100w.csv"
    CHUNK_SIZE = 500000
    test_name = ORI_TRAIN_NAMES
else:
    base_prefix = "online_"
    train_file = "/home/kesci/work/inputs/train_first_5000w.csv"
    test_file = "/home/kesci/work/inputs/test_final_part1.csv"
    CHUNK_SIZE = 50000000 # 256*1024*1024
    test_name = ORI_TEST_NAMES
print("OFFLINE:", OFFLINE)

OFFLINE: True


## 公共函数

In [2]:
ratio_func = lambda a, b: try_divide(a, b)
diff_func = lambda a, b: abs(a-b)
avg_func = lambda a, b: (a+b)/2
max_func = lambda a, b: max(a, b)
min_func = lambda a, b: min(a, b)
ngram_funcs = {
    "unichars": ngram_utils.unichars,
    "bichars": ngram_utils.bichars,
    "trichars": ngram_utils.trichars,
    "unigrams": ngram_utils.unigrams,
    "bigrams": ngram_utils.bigrams,
    "trigrams": ngram_utils.trigrams,
}

## 文本挖掘特征

In [4]:
from fuzzywuzzy import fuzz
feature_save_dir = base_feature_save_dir + "textMining/"
train_feature_prefix = base_prefix + "trainTextMining"
test_feature_prefix = base_prefix + "testTextMining"
! ls -alh /home/kesci/work/common/features/textMining/

total 6.3G
drwxr-xr-x 2 kesci root  4.0K Aug  4 03:33 .
drwxr-xr-x 7 kesci root  4.0K Aug  4 03:00 ..
-rw-r--r-- 1 kesci users  12M Aug  2 14:57 debug_testTextMining_editSim.h5
-rw-r--r-- 1 kesci users 2.7M Aug  2 13:50 debug_testTextMining_idfWordShare.h5
-rw-r--r-- 1 kesci users  12M Aug  2 14:13 debug_testTextMining_len.h5
-rw-r--r-- 1 kesci users  15M Aug  4 03:33 debug_testTextMining_ngramSim.h5
-rw-r--r-- 1 kesci users 1.9M Aug  2 13:35 debug_testTextMining_normalWordShare.h5
-rw-r--r-- 1 kesci users 1.9M Aug  2 13:11 debug_testTextMining_wordShare.h5
-rw-r--r-- 1 kesci users  29M Aug  2 14:55 debug_trainTextMining_editSim.h5
-rw-r--r-- 1 kesci users  15M Jul 23 08:47 debug_trainTextMining_editSim_xp.h5
-rw-r--r-- 1 kesci users 724K Jul 20 06:54 debug_trainTextMining_hashSim.h5
-rw-r--r-- 1 kesci users 6.8M Aug  2 13:49 debug_trainTextMining_idfWordShare.h5
-rw-r--r-- 1 kesci users  29M Aug  2 14:10 debug_trainTextMining_len.h5
-rw-r--r-- 1 kesci users 6.0M Jul 22 09:26 debug_tra

In [41]:
# AnalysisHDF("/home/kesci/work/common/features/textMining/online_trainTextMining_editSim.h5")

In [42]:
# AnalysisHDF("/home/kesci/work/common/features/textMining/online_trainTextMining_ngramSim.h5")

### 提取长度特征

In [5]:
regen_len_feature = True
feature_name = "len"

In [6]:
def run_text_len(df, ngram, prefix):
    with Timer("build {}".format(ngram)):
        if ngram == "text":
            q_list = df['query']
            t_list = df['title']
        else:
            q_list = FuncMap1(ngram_funcs[ngram], df['query'])
            t_list = FuncMap1(ngram_funcs[ngram], df['title'])
    with Timer("extract length"):
        q_len = FuncMap1(len, q_list)
        t_len = FuncMap1(len, t_list)
        del q_list, t_list
        gc.collect()
        df['%s_qLen' % prefix] = q_len
        df['%s_tLen' % prefix] = t_len
        df['%s_qtLenRatio'%prefix] = FuncMap2(ratio_func, q_len, t_len)
        df['%s_tqLenRatio'%prefix] = FuncMap2(ratio_func, t_len, q_len)
        df['%s_qtMax'%prefix] = FuncMap2(max_func, q_len, t_len)
        df['%s_qtMin'%prefix] = FuncMap2(min_func, q_len, t_len)
        df['%s_qtDiff'%prefix] = FuncMap2(diff_func, q_len, t_len)
        df['%s_qtAvg'%prefix] = FuncMap2(avg_func, q_len, t_len)
        df['%s_diffRatio'%prefix] = FuncMap2(ratio_func, df['%s_qtMin'%prefix], df['%s_qtMax'%prefix])
        del q_len, t_len
        gc.collect()
    return df
def process_text_len(df, save_dir, prefix, feature_name):
    df = run_text_len(df, "text", '%s_%s' % (prefix, 'text'))
    df = run_text_len(df, "unigrams", '%s_%s' % (prefix, 'unigrams'))
    return df

In [7]:
# 提取训练数据特征
if regen_len_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_text_len, names=train_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->started 'extract len feature' block...
----->started 'build text' block...
----->finished 'build text' block, time used:0.0s.
----->started 'extract length' block...
----->finished 'extract length' block, time used:14.99s.
----->started 'build unigrams' block...
----->finished 'build unigrams' block, time used:16.49s.
----->started 'extract length' block...
----->finished 'extract length' block, time used:16.55s.
----->started 'save feature' block...
saved len feature to /home/kesci/work/common/features/textMining/debug_trainTextMining_len.h5
----->finished 'save feature' block, time used:50.68s.
----->finished 'extract len feature' block, time used:106.42s.


In [11]:
# AnalysisHDF("/home/kesci/work/common/features/textMining/debug_trainTextMining_len.h5")

In [8]:
if regen_len_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
    process_func=process_text_len, names=test_name, process_chunkly=False, 
    chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'],
    drop_last_cols=['query', 'title'])

----->started 'extract len feature' block...
----->started 'build text' block...
----->finished 'build text' block, time used:0.0s.
----->started 'extract length' block...
----->finished 'extract length' block, time used:3.63s.
----->started 'build unigrams' block...
----->finished 'build unigrams' block, time used:4.19s.
----->started 'extract length' block...
----->finished 'extract length' block, time used:3.99s.
----->started 'save feature' block...
saved len feature to /home/kesci/work/common/features/textMining/debug_testTextMining_len.h5
----->finished 'save feature' block, time used:12.36s.
----->finished 'extract len feature' block, time used:26.0s.


### word share

In [9]:
regen_word_share = True
word_share_mode = "normal"  # normal or idf
feature_name = "{}WordShare".format(word_share_mode)

In [10]:
if word_share_mode == "idf":
    with open(tfidf_model_file, "rb") as ff:
        tfidf_model = pickle.load(ff)
        idf_ = tfidf_model.idf_
        voc_ = tfidf_model.vocabulary_
        del tfidf_model
        gc.collect()

In [11]:
def get_idf(word):
    return idf_[voc_[word]] if word in voc_ else 0.0
def stat_word_count(words):
    stat = {}
    for _ in words:
        stat[_] = stat.get(_, 0) + 1
    return stat
def stat_share_word(words1, words2):
    if word_share_mode == "normal":
        tol = sum(words1.values()) + sum(words2.values())
    elif word_share_mode == "idf":
        tol = sum(words1[w] * get_idf(w) for w in words1) + sum(words2[w] * get_idf(w) for w in words2)
    if tol < 1e-6:
        return 0.0
    if word_share_mode == "normal":
        n_shared_word_in_q1 = sum([words1[w] for w in words1 if w in words2])
        n_shared_word_in_q2 = sum([words2[w] for w in words2 if w in words1])
    elif word_share_mode == "idf":
        n_shared_word_in_q1 = sum([words1[w] * get_idf(w) for w in words1 if w in words2])
        n_shared_word_in_q2 = sum([words2[w] * get_idf(w) for w in words2 if w in words1])
    return (n_shared_word_in_q1 + n_shared_word_in_q2) / tol

In [12]:
def run_word_share(df, ngram, prefix):
    with Timer("build {}".format(ngram)):
        q_list = FuncMap1(ngram_funcs[ngram], df['query'])
        t_list = FuncMap1(ngram_funcs[ngram], df['title'])
    with Timer("extract feature"):
        q_count = FuncMap1(stat_word_count, q_list)
        t_count = FuncMap1(stat_word_count, t_list)
        del q_list, t_list
        gc.collect()
        df['{}_{}_{}'.format(prefix, feature_name, word_share_mode)] = FuncMap2(stat_share_word, q_count, t_count)
    return df
    
def process_word_share(df, save_dir, prefix, feature_name):
    df = run_word_share(df, "unigrams", '%s_%s' % (prefix, 'unigrams'))
    return df

In [13]:
# 提取训练数据特征
if regen_word_share:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_word_share, names=train_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->started 'extract normalWordShare feature' block...
----->started 'build unigrams' block...
----->finished 'build unigrams' block, time used:16.64s.
----->started 'extract feature' block...
----->finished 'extract feature' block, time used:31.08s.
----->started 'save feature' block...
saved normalWordShare feature to /home/kesci/work/common/features/textMining/debug_trainTextMining_normalWordShare.h5
----->finished 'save feature' block, time used:4.68s.
----->finished 'extract normalWordShare feature' block, time used:62.38s.


In [3]:
# !rm /home/kesci/work/common/features/textMining/debug_trainTextMining_wordShare.h5

In [14]:
# 提取训练数据特征
if regen_word_share:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_word_share, names=test_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->started 'extract normalWordShare feature' block...
----->started 'build unigrams' block...
----->finished 'build unigrams' block, time used:4.21s.
----->started 'extract feature' block...
----->finished 'extract feature' block, time used:7.71s.
----->started 'save feature' block...
saved normalWordShare feature to /home/kesci/work/common/features/textMining/debug_testTextMining_normalWordShare.h5
----->finished 'save feature' block, time used:1.13s.
----->finished 'extract normalWordShare feature' block, time used:15.26s.


### 提取字符编辑距离特征

In [15]:
import Levenshtein
regen_edit_sim_feature = True
feature_name = "editSim"

In [16]:
similar_arr = [
    # https://www.jb51.net/article/98449.htm;
    # http://www.coli.uni-saarland.de/courses/LT1/2011/slides/Python-Levenshtein.html#Levenshtein-inverse
    Levenshtein.distance,
    Levenshtein.jaro,
    Levenshtein.jaro_winkler,
    # https://blog.csdn.net/qq_43174128/article/details/82595317
    fuzz.ratio,
    fuzz.partial_ratio,
    fuzz.token_sort_ratio,
    fuzz.partial_token_sort_ratio,
    fuzz.token_set_ratio,
    # dist_utils.edit_dist,
    # dist_utils.is_str_match,
    dist_utils.longest_match_size,
    dist_utils.longest_match_ratio,
    # dist_utils.compression_dist
]

In [17]:
def run_edit_sim_feature(df, prefix):
    q_list = df['query']
    t_list = df['title']
    with Timer("extract edit sim"):
        for _ in similar_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = FuncMap2(_, q_list, t_list)
        del q_list, t_list
        gc.collect()
    return df
def process_edit_sim_feature(df, save_dir, prefix, feature_name):
    df = run_edit_sim_feature(df, '%s_%s' % (prefix, 'text'))
    return df

In [18]:
if regen_edit_sim_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_edit_sim_feature, names=train_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->started 'extract editSim feature' block...
----->started 'extract edit sim' block...
----->started 'cal distance sim' block...
----->finished 'cal distance sim' block, time used:11.56s.
----->started 'cal jaro sim' block...
----->finished 'cal jaro sim' block, time used:4.66s.
----->started 'cal jaro_winkler sim' block...
----->finished 'cal jaro_winkler sim' block, time used:4.75s.
----->started 'cal ratio sim' block...
----->finished 'cal ratio sim' block, time used:23.46s.
----->started 'cal partial_ratio sim' block...
----->finished 'cal partial_ratio sim' block, time used:126.22s.
----->started 'cal token_sort_ratio sim' block...
----->finished 'cal token_sort_ratio sim' block, time used:94.28s.
----->started 'cal partial_token_sort_ratio sim' block...
----->finished 'cal partial_token_sort_ratio sim' block, time used:202.95s.
----->started 'cal token_set_ratio sim' block...
----->finished 'cal token_set_ratio sim' block, time used:136.93s.
----->started 'cal longest_match_s

In [36]:
# AnalysisCSV("/home/kesci/work/common/features/textMining/debug_trainTextMining_editSim.csv.gz", print_rows=10)

In [19]:
if regen_edit_sim_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_edit_sim_feature, names=test_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', "label"], 
        drop_last_cols=['query', 'title'])

----->started 'extract editSim feature' block...
----->started 'extract edit sim' block...
----->started 'cal distance sim' block...
----->finished 'cal distance sim' block, time used:2.9s.
----->started 'cal jaro sim' block...
----->finished 'cal jaro sim' block, time used:1.16s.
----->started 'cal jaro_winkler sim' block...
----->finished 'cal jaro_winkler sim' block, time used:1.18s.
----->started 'cal ratio sim' block...
----->finished 'cal ratio sim' block, time used:5.9s.
----->started 'cal partial_ratio sim' block...
----->finished 'cal partial_ratio sim' block, time used:31.75s.
----->started 'cal token_sort_ratio sim' block...
----->finished 'cal token_sort_ratio sim' block, time used:23.21s.
----->started 'cal partial_token_sort_ratio sim' block...
----->finished 'cal partial_token_sort_ratio sim' block, time used:50.16s.
----->started 'cal token_set_ratio sim' block...
----->finished 'cal token_set_ratio sim' block, time used:34.19s.
----->started 'cal longest_match_size sim

### 提取集合相似度特征

In [20]:
import Levenshtein
regen_ngram_sim_feature = True
feature_name = "ngramSim"

In [21]:
ratio_arr = [
    # 集合的交集关系
    # 可以操作字符串，如"abc"；也可以操作字符数组，如['a', 'b', 'c']
    dist_utils.dice_ratio,
    dist_utils.jaccard_ratio,
    dist_utils.edit_seq_ratio,
    dist_utils.edit_set_ratio,
]

In [22]:
def run_ngram_similarity(df, ngram, prefix):
    q_list = FuncMap1(ngram_funcs[ngram], df['query'])
    t_list = FuncMap1(ngram_funcs[ngram], df['title'])
    with Timer("extract ngram sim"):
        for _ in ratio_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = FuncMap2(_, q_list, t_list)
        del q_list, t_list
        gc.collect()
    return df

def process_ngram_similarity(df, save_dir, prefix, feature_name):
    # df = run_ngram_similarity(df, 'unichars', '%s_%s' % (prefix, 'unichars'))
    # df = run_ngram_similarity(df, 'bichars', '%s_%s' % (prefix, 'bichars'))
    # df = run_ngram_similarity(df, 'trichars', '%s_%s' % (prefix, 'trichars'))
    df = run_ngram_similarity(df, 'unigrams', '%s_%s' % (prefix, 'unigrams'))
    df = run_ngram_similarity(df, 'bigrams', '%s_%s' % (prefix, 'bigrams'))
    # df = run_ngram_similarity(df, 'trigrams', '%s_%s' % (prefix, 'trigrams'))
    return df

In [23]:
# 提取训练数据特征
if regen_ngram_sim_feature:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_ngram_similarity, names=train_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

----->started 'extract ngramSim feature' block...
----->started 'extract ngram sim' block...
----->started 'cal dice_ratio sim' block...
----->finished 'cal dice_ratio sim' block, time used:10.1s.
----->started 'cal jaccard_ratio sim' block...
----->finished 'cal jaccard_ratio sim' block, time used:8.21s.
----->started 'cal edit_seq_ratio sim' block...
----->finished 'cal edit_seq_ratio sim' block, time used:22.47s.
----->started 'cal edit_set_ratio sim' block...
----->finished 'cal edit_set_ratio sim' block, time used:26.72s.
----->finished 'extract ngram sim' block, time used:70.31s.
----->started 'extract ngram sim' block...
----->started 'cal dice_ratio sim' block...
----->finished 'cal dice_ratio sim' block, time used:9.69s.
----->started 'cal jaccard_ratio sim' block...
----->finished 'cal jaccard_ratio sim' block, time used:7.73s.
----->started 'cal edit_seq_ratio sim' block...
----->finished 'cal edit_seq_ratio sim' block, time used:30.95s.
----->started 'cal edit_set_ratio sim

In [41]:
# AnalysisCSV("./stage1/output/text_mining_feature/debug_trainTextMining_ngramSim.csv.gz")

In [24]:
if regen_ngram_sim_feature:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_ngram_similarity, names=test_name, process_chunkly=False, 
        chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', "label"], 
        drop_last_cols=['query', 'title'])

----->started 'extract ngramSim feature' block...
----->started 'extract ngram sim' block...
----->started 'cal dice_ratio sim' block...
----->finished 'cal dice_ratio sim' block, time used:2.51s.
----->started 'cal jaccard_ratio sim' block...
----->finished 'cal jaccard_ratio sim' block, time used:2.07s.
----->started 'cal edit_seq_ratio sim' block...
----->finished 'cal edit_seq_ratio sim' block, time used:5.57s.
----->started 'cal edit_set_ratio sim' block...
----->finished 'cal edit_set_ratio sim' block, time used:6.56s.
----->finished 'extract ngram sim' block, time used:17.4s.
----->started 'extract ngram sim' block...
----->started 'cal dice_ratio sim' block...
----->finished 'cal dice_ratio sim' block, time used:2.38s.
----->started 'cal jaccard_ratio sim' block...
----->finished 'cal jaccard_ratio sim' block, time used:1.92s.
----->started 'cal edit_seq_ratio sim' block...
----->finished 'cal edit_seq_ratio sim' block, time used:7.73s.
----->started 'cal edit_set_ratio sim' bl

### SimHash特征

In [29]:
# https://github.com/cjauvin/simhash/blob/master/tests/test_simhash.py
# https://leons.im/posts/a-python-implementation-of-simhash-algorithm/
# 这个不错: http://yanyiwu.com/work/2014/01/30/simhash-shi-xian-xiang-jie.html
from simhash import Simhash
regen_hashsim = False
feature_name = "hashSim"

In [32]:
def run_ngram_hashsim(df, prefix):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
            voc = {i:w for w, i in tfidf_model.vocabulary_.items()}
        q_vecs = tfidf_model.transform(df['query'])
        t_vecs = tfidf_model.transform(df['title'])
    with Timer("cal hash pairs"):
        q_simhash = [
            Simhash(
                zip([voc[j] for j in Di.indices], Di.data)
            )
            for Di in q_vecs
        ]
        t_simhash = [
            Simhash(
                zip([voc[j] for j in Di.indices], Di.data)
            )
            for Di in t_vecs
        ]
    with Timer("cal hash sim"):
        df["{}_hashSim".format(prefix)] = FuncMap2(lambda x,y: x.distance(y), q_simhash, t_simhash)
        
    del q_vecs, t_vecs, q_simhash, t_simhash, tfidf_model, voc
    gc.collect()
    
    return df

def process_ngram_hashsim(df, save_dir, prefix, feature_name):
    df = run_ngram_hashsim(df, prefix)
    return df

In [33]:
# 提取训练数据特征
if regen_hashsim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
                   process_func=process_ngram_hashsim, names=ORI_TRAIN_NAMES, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
                   drop_last_cols=['query', 'title'])

----->started 'extract hashSim feature' block...
----->started 'cal tfidf vec' block...
----->finished 'cal tfidf vec' block, time used:198.89s.
----->started 'cal sim hash' block...
----->finished 'cal sim hash' block, time used:1239.1s.
----->started 'cal hash sim' block...
----->finished 'cal hash sim' block, time used:3.57s.
----->started 'save feature' block...
saved hashSim feature to /home/kesci/work/common/features/textMining/debug_trainTextMining_hashSim.h5
----->finished 'save feature' block, time used:0.51s.
----->finished 'extract hashSim feature' block, time used:1576.93s.


In [35]:
# AnalysisHDF("/home/kesci/work/common/features/textMining/debug_trainTextMining_hashSim.h5", print_rows=10)

In [ ]:
# 提取训练数据特征
if regen_hashsim and not OFFLINE:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
                   process_func=process_ngram_hashsim, names=ORI_TEST_NAMES, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id'], 
                   drop_last_cols=['query', 'title'])

### 最长公共子串

In [48]:
import py_common_subseq
regen_subseq = False
feature_name = "commonSeq"
print(py_common_subseq.find_common_subsequences("123", "13423"))

{'', '23', '123', '2', '3', '1', '13', '12'}


In [49]:
def run_ngram_subseq(df, ngram, prefix):
    query = df["query"].apply(ngram)
    title = df["title"].apply(ngram)
    with Timer("cal sub seq"):
        df["{}LCSValue".format(prefix)] = FuncMap2(lambda x, y:py_common_subseq.find_common_subsequences(x, y),query, title)
        del query, title
        gc.collect()
    return df

def process_ngram_subseq(df, save_dir, prefix, feature_name):
    df = run_ngram_subseq(df, ngram_utils.unichars, '%s_%s' % (prefix, 'unichars'))
    df = run_ngram_subseq(df, ngram_utils.bichars, '%s_%s' % (prefix, 'bichars'))
    df = run_ngram_subseq(df, ngram_utils.trichars, '%s_%s' % (prefix, 'trichars'))
    df = run_ngram_subseq(df, ngram_utils.unigrams, '%s_%s' % (prefix, 'unigrams'))
    df = run_ngram_subseq(df, ngram_utils.bigrams, '%s_%s' % (prefix, 'bigrams'))
    df = run_ngram_subseq(df, ngram_utils.trigrams, '%s_%s' % (prefix, 'trigrams'))
    return df 

In [50]:
# 提取训练数据特征
if regen_subseq:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_ngram_subseq, names=ORI_TRAIN_NAMES, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

In [51]:
# 提取训练数据特征
if not OFFLINE and regen_subseq:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_ngram_subseq, names=ORI_TEST_NAMES, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], drop_last_cols=['query', 'title'])

## 向量空间特征

In [5]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
feature_save_dir = base_feature_save_dir + "vectorSpace/"
train_feature_prefix = base_prefix + "trainVectorSpace"
test_feature_prefix = base_prefix + "testVectorSpace"
print(tfidf_model_file)
! ls -alh /home/kesci/work/common/features/vectorSpace
# debug_trainVectorSpace_tfidfVecLen.h5

/home/kesci/work/common/util_models/tfidf_word_ngram-13_online_allUnique.bin
total 8.0G
drwxr-xr-x 2 kesci root  4.0K Aug  1 20:25 .
drwxr-xr-x 7 kesci root  4.0K Aug  4 03:00 ..
-rw-r--r-- 1 kesci users  34M Aug  6 13:36 debug_testVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users 132M Aug  6 13:27 debug_trainVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users  52M Jul 21 08:02 debug_trainVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users  19M Jul 22 11:26 debug_trainVectorSpace_tfidfVecSim.h5
-rw-r--r-- 1 kesci users  20M Jul 22 03:09 debug_trainVectorSpace_tfidfVecSim_xp.h5
-rw-r--r-- 1 kesci users 894M Aug  1 20:30 online_testVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users 980M Jul 24 04:13 online_testVectorSpace_tfidfVecLen.h5
-rw-r--r-- 1 kesci users 373M Jul 24 01:58 online_testVectorSpace_tfidfVecSim.h5
-rw-r--r-- 1 kesci users 2.2G Aug  1 19:14 online_trainVectorSpace_tfidfVecFeat.h5
-rw-r--r-- 1 kesci users 2.5G Jul 24 03:33 online_trainVectorSpace_tfidfVecLen.h5
-rw

### tfidf向量相似性

In [4]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
from sklearn.metrics.pairwise import *
regen_tfidf_vec_sim = True
feature_name = "tfidfVecFeat"
print(tfidf_model_file)

/home/kesci/work/common/util_models/tfidf_word_ngram-13_online_allUnique.bin


In [5]:
dis_arr = [
    paired_cosine_distances,
    paired_euclidean_distances,
    paired_manhattan_distances
]

In [7]:
def process_tfidf_vec_feat(df, save_dir, prefix, feature_name):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec = tfidf_model.transform(df["query"])
        t_vec = tfidf_model.transform(df["title"])
    with Timer("cal tfidf vec value"):
        df['{}_{}_QSum'.format(prefix, feature_name)] = np.array(q_vec.sum(1)).squeeze()
        df['{}_{}_TSum'.format(prefix, feature_name)] = np.array(t_vec.sum(1)).squeeze()
        # df['{}_{}_QMean'.format(prefix, feature_name)] = np.array(q_vec.mean(1)).squeeze()
        # df['{}_{}_TMean'.format(prefix, feature_name)] = np.array(t_vec.mean(1)).squeeze()
        df['{}_{}_QLen'.format(prefix, feature_name)] = [len(Di.data) for Di in q_vec]
        df['{}_{}_TLen'.format(prefix, feature_name)] = [len(Di.data) for Di in t_vec]
    with Timer("cal tfidf vec sim"):
        for _ in dis_arr:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}_{}".format(prefix, feature_name, name)] = _(q_vec, t_vec)
        del q_vec, t_vec, tfidf_model
        gc.collect()
    return df

In [8]:
if regen_tfidf_vec_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
                  process_func=process_tfidf_vec_feat, names=train_name, process_chunkly=False, 
                  chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
                  drop_last_cols=['query', 'title'])

----->started 'extract tfidfVecFeat feature' block...
----->started 'cal tfidf vec' block...
----->finished 'cal tfidf vec' block, time used:421.39s.
----->started 'cal tfidf vec value' block...
----->finished 'cal tfidf vec value' block, time used:379.11s.
----->started 'cal tfidf vec sim' block...
----->started 'cal paired_cosine_distances sim' block...
----->finished 'cal paired_cosine_distances sim' block, time used:8.93s.
----->started 'cal paired_euclidean_distances sim' block...
----->finished 'cal paired_euclidean_distances sim' block, time used:7.85s.
----->started 'cal paired_manhattan_distances sim' block...
----->finished 'cal paired_manhattan_distances sim' block, time used:8.0s.
----->finished 'cal tfidf vec sim' block, time used:200.43s.
----->started 'save feature' block...
saved tfidfVecFeat feature to /home/kesci/work/common/features/vectorSpace/debug_trainVectorSpace_tfidfVecFeat.h5
----->finished 'save feature' block, time used:40.01s.
----->finished 'extract tfidfV

In [9]:
if regen_tfidf_vec_sim:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
                   process_func=process_tfidf_vec_feat, names=test_name, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', 'label'], 
                   drop_last_cols=['query', 'title'])

----->started 'extract tfidfVecFeat feature' block...
----->started 'cal tfidf vec' block...
----->finished 'cal tfidf vec' block, time used:253.96s.
----->started 'cal tfidf vec value' block...
----->finished 'cal tfidf vec value' block, time used:94.76s.
----->started 'cal tfidf vec sim' block...
----->started 'cal paired_cosine_distances sim' block...
----->finished 'cal paired_cosine_distances sim' block, time used:2.73s.
----->started 'cal paired_euclidean_distances sim' block...
----->finished 'cal paired_euclidean_distances sim' block, time used:2.31s.
----->started 'cal paired_manhattan_distances sim' block...
----->finished 'cal paired_manhattan_distances sim' block, time used:2.51s.
----->finished 'cal tfidf vec sim' block, time used:224.99s.
----->started 'save feature' block...
saved tfidfVecFeat feature to /home/kesci/work/common/features/vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5
----->finished 'save feature' block, time used:9.04s.
----->finished 'extract tfidfVec

In [11]:
AnalysisHDF("/home/kesci/work/common/features/vectorSpace/debug_testVectorSpace_tfidfVecFeat.h5")

--->file shape:
(400000, 9)
--->2 rows of file:
   debug_testVectorSpace_tfidfVecFeat_QSum  \
0                                 1.913374   
1                                 1.913374   

   debug_testVectorSpace_tfidfVecFeat_TSum  \
0                                 5.618418   
1                                 4.642403   

   debug_testVectorSpace_tfidfVecFeat_QMean  \
0                              3.504090e-08   
1                              3.504090e-08   

   debug_testVectorSpace_tfidfVecFeat_TMean  \
0                              1.028939e-07   
1                              8.501944e-08   

   debug_testVectorSpace_tfidfVecFeat_QLen  \
0                                        4   
1                                        4   

   debug_testVectorSpace_tfidfVecFeat_TLen  \
0                                       36   
1                                       26   

   debug_testVectorSpace_tfidfVecFeat_paired_cosine_distances  \
0                                           0.7

### tfidf值特征

In [9]:
regen_tfidf_vec_len = True
feature_name = "tfidfVecLen"
print(tfidf_model_file)

In [10]:
def process_tfidf_vec_len(df, save_dir, prefix, feature_name):
    with Timer("cal tfidf vec"):
        with open(tfidf_model_file, "rb") as ff:
            tfidf_model = pickle.load(ff)
        q_vec = tfidf_model.transform(df["query"])
        t_vec = tfidf_model.transform(df["title"])
    with Timer("cal tfidf len"):
        q_vec_sum = np.array(q_vec.sum(1)).squeeze()
        t_vec_sum = np.array(t_vec.sum(1)).squeeze()
        q_vec_mean = np.array(q_vec.mean(1)).squeeze()
        t_vec_mean = np.array(t_vec.mean(1)).squeeze()
        df['%s_%s_QSum' % (feature_name, prefix)] = q_vec_sum
        df['%s_%s_TSum' % (feature_name, prefix)] = t_vec_sum
        # df['%s_%s_QMean' % (feature_name, prefix)] = q_vec_mean
        # df['%s_%s_TMean' % (feature_name, prefix)] = t_vec_mean
        # df['%s_QTLenRatio'%prefix] = FuncMap2(ratio_func, q_vec_len, t_vec_len)
        # df['%s_TQLenRatio'%prefix] = FuncMap2(ratio_func, t_vec_len, q_vec_len)
        # df['%s_QTDiff'%prefix] = FuncMap2(diff_func, q_vec_len, t_vec_len)
        # df['%s_QTMax'%prefix] = FuncMap2(max_func, q_vec_len, t_vec_len)
        # df['%s_QTMin'%prefix] = FuncMap2(min_func, q_vec_len, t_vec_len)
        # df['%s_QTAvg'%prefix] = FuncMap2(avg_func, q_vec_len, t_vec_len)
        # df['%s_QTMulti'%prefix] = FuncMap2(lambda a, b: a*b, q_vec_len, t_vec_len)
        # df['%s_QSquare2'%prefix] = numpy.square(q_vec_len)
        # df['%s_TSquare2'%prefix] = numpy.square(t_vec_len)
        # df['%s_QSqrt'%prefix] = numpy.sqrt(q_vec_len)
        # df['%s_TSqrt'%prefix] = numpy.sqrt(t_vec_len)
        # df['%s_QLog'%prefix] = numpy.log(q_vec_len)
        # df['%s_TLog'%prefix] = numpy.log(t_vec_len)
        # p_corr = scipy.stats.pearsonr(q_vec_len, t_vec_len)[0]
        # df['%s_PCorr'%prefix] = [p_corr] * len(q_vec_len)
        del q_vec, t_vec, q_vec_sum, t_vec_sum, q_vec_mean, t_vec_mean, tfidf_model
        gc.collect()
    return df

In [11]:
if regen_tfidf_vec_len:
    ExtractFeature1(train_csv=train_file, test_csv=test_file, 
        train_name=train_name, test_name=test_name, 
        process_func=process_tfidf_vec_len, 
        save_dir=feature_save_dir, 
        prefix=feature_prefix, 
        feature_name=feature_name)

## 提取word2vec特征

In [6]:
import pickle
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from scipy.stats import skew, kurtosis
feature_save_dir = base_feature_save_dir + "word2vec/"
train_feature_prefix = base_prefix + "trainWord2vec"
test_feature_prefix = base_prefix + "testWord2vec"
! ls -alh /home/kesci/work/common/features/word2vec

total 1.4G
drwxr-xr-x 2 kesci root  4.0K Aug  5 21:51 .
drwxr-xr-x 7 kesci root  4.0K Aug  4 03:00 ..
-rw-r--r-- 1 kesci users 5.7M Aug  2 04:09 debug_testWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 2.7M Aug  4 05:30 debug_testWord2vec_tfidfSenVecSim.h5
-rw-r--r-- 1 kesci users  14M Aug  2 04:07 debug_trainWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 6.7M Aug  4 05:21 debug_trainWord2vec_tfidfSenVecSim.h5
-rw-r--r-- 1 kesci users 264M Aug  2 02:41 online_testWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 130M Aug  5 21:51 online_testWord2vec_tfidfSenVecSim.h5
-rw-r--r-- 1 kesci users 649M Aug  2 00:53 online_trainWord2vec_avgSenVecSim.h5
-rw-r--r-- 1 kesci users 323M Aug  5 19:57 online_trainWord2vec_tfidfSenVecSim.h5


In [26]:
def get_idf(word):
    return tfidf_idf[tfidf_vocab[word]] if word in tfidf_vocab else 0.0
def sent2vec1(text):
    global word2vec_model
    word_count = {}
    for _ in text.split():
        word_count[_] = word_count.get(_, 0) + 1
    return np.nan_to_num(
          np.array([np.array(word2vec_model[w])*get_idf(w)*word_count[_] for w in word_count if w in word2vec_model] or [0.0] * word2vec_embed_size).mean(axis=0)
      )

In [7]:
def sent2vec(text):
    global word2vec_model
    # TODO: 只能计算unigrams的值
    return np.nan_to_num(
          np.array([word2vec_model[w] for w in text.split() if w in word2vec_model] or [0.0] * word2vec_embed_size
          ).mean(axis=0)
      )

### 生成word2vec句子相似度

In [8]:
# word2vec to sent: https://www.zhihu.com/question/29978268
regen_word2vec_sen_vec_sim = True
process_word2vec_sen_vec_mode = "avg"  # "avg" or "tfidf"
feature_name = "{}SenVecSim".format(process_word2vec_sen_vec_mode)

In [9]:
similar_dis = [
    dist_utils.cosine_distance, 
    dist_utils.euclidean_distance,
    dist_utils.cityblock_distance,
    # dist_utils.jaccard_distance,
    # dist_utils.braycurtis_distance,
    # dist_utils.canberra_distance,
    # dist_utils.minkowski_distance
]

In [10]:
if regen_word2vec_sen_vec_sim:
    word2vec_model = KeyedVectors.load(word2vec_kv_model_file, mmap='r')

/opt/conda/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [31]:
if regen_word2vec_sen_vec_sim and process_word2vec_sen_vec_mode == "tfidf":
    with open(tfidf_model_file, "rb") as ff:
        tfidf_model = pickle.load(ff)
    tfidf_idf = tfidf_model.idf_
    tfidf_vocab = tfidf_model.vocabulary_

In [11]:
def process_word2vec_sen_vec(df, save_dir, prefix, feature_name):
    with Timer("cal sent vec"):
        if process_word2vec_sen_vec_mode == "avg":
            q_sen_vec = FuncMap1(sent2vec, df["query"])
            t_sen_vec = FuncMap1(sent2vec, df["title"])
        elif process_word2vec_sen_vec_mode == "tfidf":
            q_sen_vec = FuncMap1(sent2vec1, df["query"])
            t_sen_vec = FuncMap1(sent2vec1, df["title"])
    with Timer("cal sent vec sim"):
        for _ in similar_dis:
            name = _.__name__
            with Timer("cal {} sim".format(name)):
                df["{}_{}".format(prefix, name)] = FuncMap2(_, q_sen_vec, t_sen_vec)
        del q_sen_vec, t_sen_vec
        gc.collect()
    return df

In [12]:
if regen_word2vec_sen_vec_sim:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
                   process_func=process_word2vec_sen_vec, names=train_name, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', "label"], 
                   drop_last_cols=['query', 'title'])

----->started 'extract avgSenVecSim feature' block...
----->started 'cal sent vec' block...
----->finished 'cal sent vec' block, time used:683.36s.
----->started 'cal sent vec sim' block...
----->started 'cal cosine_distance sim' block...
----->finished 'cal cosine_distance sim' block, time used:158.8s.
----->started 'cal euclidean_distance sim' block...
----->finished 'cal euclidean_distance sim' block, time used:173.26s.
----->started 'cal cityblock_distance sim' block...
----->finished 'cal cityblock_distance sim' block, time used:129.03s.
----->finished 'cal sent vec sim' block, time used:463.35s.
----->started 'save feature' block...
saved avgSenVecSim feature to /home/kesci/work/common/features/word2vec/debug_trainWord2vec_avgSenVecSim.h5
----->finished 'save feature' block, time used:32.64s.
----->finished 'extract avgSenVecSim feature' block, time used:1188.21s.


In [13]:
if regen_word2vec_sen_vec_sim:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
                   process_func=process_word2vec_sen_vec, names=test_name, process_chunkly=False, 
                   chunk_size=CHUNK_SIZE, drop_first_cols=['query_id', 'query_title_id', "label"], 
                   drop_last_cols=['query', 'title'])

----->started 'extract avgSenVecSim feature' block...
----->started 'cal sent vec' block...
----->finished 'cal sent vec' block, time used:167.04s.
----->started 'cal sent vec sim' block...
----->started 'cal cosine_distance sim' block...
----->finished 'cal cosine_distance sim' block, time used:39.84s.
----->started 'cal euclidean_distance sim' block...
----->finished 'cal euclidean_distance sim' block, time used:43.86s.
----->started 'cal cityblock_distance sim' block...
----->finished 'cal cityblock_distance sim' block, time used:32.43s.
----->finished 'cal sent vec sim' block, time used:116.85s.
----->started 'save feature' block...
saved avgSenVecSim feature to /home/kesci/work/common/features/word2vec/debug_testWord2vec_avgSenVecSim.h5
----->finished 'save feature' block, time used:7.95s.
----->finished 'extract avgSenVecSim feature' block, time used:293.33s.


### word2vec其他特征

In [9]:
word2vec_model_norm = None
regen_word2vec_sen_vec_other = False
process_word2vec_sen_vec_mode = "avg"
feature_name = "{}SenVecOther".format(process_word2vec_sen_vec_mode)

In [111]:
def norm_wmd(s1, s2):
    global word2vec_model_norm
    dis = np.nan_to_num(word2vec_model_norm.wmdistance(s1, s2))
    return dis if dis<100 else 100

In [112]:
def wmd(s1, s2):
    global word2vec_model
    dis = np.nan_to_num(word2vec_model.wmdistance(s1, s2))
    return dis if dis<100 else 100

In [113]:
def run_word2vec_sen_vec_wmd(df, ngram, prefix, name):
    q_list = df['query'].apply(ngram)
    t_list = df['title'].apply(ngram)
    df['%s_wmd%s' % (prefix, name)] = FuncMap2(wmd, q_list, t_list)
    df['%s_normWmd%s' % (prefix, name)] = FuncMap2(norm_wmd, q_list, t_list)
    del q_list, t_list
    gc.collect()
    return df

In [114]:
def run_word2vec_sen_vec_sk(df, func, q_sen_vec, t_sen_vec, prefix, name):
    q_value = FuncMap1(func, q_sen_vec)
    t_value = FuncMap1(func, t_sen_vec)
    df['%s_q%s' % (prefix, name)] = q_value
    df['%s_t%s' % (prefix, name)] = t_value
    df['%s_%sDiff'%(prefix, name)] = FuncMap2(lambda a, b: abs(a-b), q_value, t_value)
    df['%s_%sMax'%(prefix, name)] = FuncMap2(lambda a, b: max(a, b), q_value, t_value)
    df['%s_%sMin'%(prefix, name)] = FuncMap2(lambda a, b: min(a, b), q_value, t_value)
    df['%s_%sAvg'%(prefix, name)] = FuncMap2(lambda a, b: (a+b)/2, q_value, t_value)
    return df

In [115]:
def process_word2vec_sen_vec_other(df, save_dir, prefix, feature_name):
    
    global word2vec_model_norm, word2vec_model_file
    word2vec_model_norm = KeyedVectors.load(word2vec_model_file)
    word2vec_model_norm.init_sims(replace=True)
    
    # with Timer("cal uni-wmd dis"):
    #     df = run_word2vec_sen_vec_wmd(df, ngram_utils.unigrams, prefix, "Unigrams")
    # with Timer("cal bi-wmd dis"):
    #     df = run_word2vec_sen_vec_wmd(df, ngram_utils.bigrams, prefix, "Bigrams")
        
    with Timer("cal sent vec"):
        global process_word2vec_sen_vec_mode
        if process_word2vec_sen_vec_mode == "avg":
            q_sen_vec = FuncMap1(sent2vec, df["query"])
            t_sen_vec = FuncMap1(sent2vec, df["title"])
        elif process_word2vec_sen_vec_mode == "tfidf":
            q_sen_vec = FuncMap1(sent2vec1, df["query"])
            t_sen_vec = FuncMap1(sent2vec1, df["title"])
    
    with Timer("cal skew"):
        df = run_word2vec_sen_vec_sk(df, skew, q_sen_vec, t_sen_vec, prefix, "Skew")
    with Timer("cal kurtosis"):
        df = run_word2vec_sen_vec_sk(df, kurtosis, q_sen_vec, t_sen_vec, prefix, "Kurtosis")
        
    del q_sen_vec, t_sen_vec
    gc.collect()
    
    return df

In [117]:
if regen_word2vec_sen_vec_other:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name,
        process_func=process_word2vec_sen_vec_other, names=ORI_TRAIN_NAMES,
        process_chunkly=True, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_id', 'query_title_id', 'label'], 
        drop_last_cols=['query', 'title'])

## Magic特征

In [22]:
feature_save_dir = base_feature_save_dir + "magic/"
train_feature_prefix = base_prefix + "trainMagic"
test_feature_prefix = base_prefix + "testMagic"
! ls -alh /home/kesci/work/common/features/magic

total 63M
drwxr-xr-x 2 kesci root  4.0K Jul 23 15:40 .
drwxr-xr-x 6 kesci root  4.0K Jul 22 01:47 ..
-rw-r--r-- 1 kesci users 716K Jul 22 11:57 debug_trainMagic_localCTR.h5
-rw-r--r-- 1 kesci users  19M Jul 23 15:40 debug_trainMagic_localTfidfSim.h5
-rw-r--r-- 1 kesci users 217K Jul 20 14:39 debug_trainMagic_queryFreq.h5
-rw-r--r-- 1 kesci users  12M Jul 22 01:37 online_testMagic_localCTR.h5
-rw-r--r-- 1 kesci users 2.4M Jul 22 01:34 online_testMagic_queryFreq.h5
-rw-r--r-- 1 kesci users  24M Jul 22 01:36 online_trainMagic_localCTR.h5
-rw-r--r-- 1 kesci users 5.0M Jul 22 01:33 online_trainMagic_queryFreq.h5


### 提取query频率

In [3]:
regen_query_freq = True
feature_name = "queryFreq"

In [4]:
def process_query_freq(df, save_dir, prefix, feature_name):
    grouped = df.groupby("query_id", as_index=False)["query_title_id"].count()
    res = np.concatenate(FuncMap1(lambda x: [x]*x, grouped["query_title_id"]))
    del grouped, df
    gc.collect()
    return pd.DataFrame({
        feature_name: res
    })

In [5]:
if regen_query_freq:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_query_freq, names=ORI_TRAIN_NAMES,
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query', 'title', 'label'], 
        drop_last_cols=['query_id', 'query_title_id'], strip=False)

----->started 'extract queryFreq feature' block...
----->started 'save feature' block...
saved queryFreq feature to /home/kesci/work/common/features/magic/online_trainMagic_queryFreq.h5
----->finished 'save feature' block, time used:26.75s.
----->finished 'extract queryFreq feature' block, time used:122.85s.


In [103]:
# AnalysisHDF("/home/kesci/work/common/features/magic/debug_trainMagic_queryFreq.h5", print_rows=10)

In [6]:
if regen_query_freq and not OFFLINE:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_query_freq, names=ORI_TEST_NAMES,
        process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query', 'title'], 
        drop_last_cols=['query_id', 'query_title_id'],strip=False)

----->started 'extract queryFreq feature' block...
----->started 'save feature' block...
saved queryFreq feature to /home/kesci/work/common/features/magic/online_testMagic_queryFreq.h5
----->finished 'save feature' block, time used:11.87s.
----->finished 'extract queryFreq feature' block, time used:51.33s.


### 局部tfidf特征

In [39]:
regen_localTfidf = True
feature_name = "localTfidf"
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import *

In [40]:
dis_arr = [
    paired_cosine_distances,
    paired_euclidean_distances,
    paired_manhattan_distances
]

In [41]:
def process_local_tfidf(df, save_dir, prefix, feature_name):
    grouped = df.groupby("query_id")
    dis_res = {}
    for _ in dis_arr:
        dis_res[_.__name__] = []
    all_q_vec_len, all_t_vec_len = [], []
    with Timer("cal local tfidf vec"):
        for name, group in grouped:
            word_tfidf = TfidfVectorizer(norm="l2",  # 'l2 norm' can cal similarty
                                strip_accents="unicode",
                                analyzer="word",
                                ngram_range=(1, 3),
                                use_idf=True,
                                smooth_idf=True,
                                sublinear_tf=True, min_df=1, max_df=1.0)
            corpus = [_ for _ in group["title"]]
            corpus.append(group["query"].iloc[-1])
            word_tfidf.fit(corpus)
            q_vec = word_tfidf.transform(group["query"])
            t_vec = word_tfidf.transform(group["title"])
            for _ in dis_arr:
                dis_res[_.__name__].append(_(q_vec, t_vec))
            all_q_vec_len.append(np.array(q_vec.sum(1)).squeeze())
            all_t_vec_len.append(np.array(t_vec.sum(1)).squeeze())
    with Timer("cal sim"):
        for _ in dis_arr:
            name = _.__name__
            df["{}_{}".format(prefix, name)] = np.concatenate(dis_res[name])
        del dis_res
        gc.collect()
    with Timer("cal len"):
        q_vec_len = np.concatenate(all_q_vec_len)
        t_vec_len = np.concatenate(all_t_vec_len)
        df['%s_QLen' % prefix] = q_vec_len
        df['%s_TLen' % prefix] = t_vec_len
        df['%s_QTLenRatio'%prefix] = FuncMap2(ratio_func, q_vec_len, t_vec_len)
        df['%s_TQLenRatio'%prefix] = FuncMap2(ratio_func, t_vec_len, q_vec_len)
        df['%s_QTDiff'%prefix] = FuncMap2(diff_func, q_vec_len, t_vec_len)
        df['%s_QTMax'%prefix] = FuncMap2(max_func, q_vec_len, t_vec_len)
        df['%s_QTMin'%prefix] = FuncMap2(min_func, q_vec_len, t_vec_len)
        df['%s_QTAvg'%prefix] = FuncMap2(avg_func, q_vec_len, t_vec_len)
        del all_q_vec_len, all_t_vec_len, q_vec_len, t_vec_len
    return df

In [35]:
if regen_localTfidf:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
        process_func=process_local_tfidf, 
        names=ORI_TRAIN_NAMES, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_title_id', 'label'], drop_last_cols=['query_id', 'query', 'title'])

----->started 'extract localTfidf feature' block...
----->started 'cal local tfidf vec' block...
----->finished 'cal local tfidf vec' block, time used:13410.16s.
----->finished 'extract localTfidf feature' block, time used:13520.84s.


KeyboardInterrupt: 

In [10]:
# AnalysisHDF("/home/kesci/work/common/features/magic/debug_trainMagic_localTfidfSim.h5")

In [ ]:
if regen_localTfidf and not OFFLINE:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
        process_func=process_local_tfidf, 
        names=ORI_TEST_NAMES, process_chunkly=False, chunk_size=CHUNK_SIZE, 
        drop_first_cols=['query_title_id'], drop_last_cols=['query_id', 'query', 'title'])

### 局部点击率特征

In [22]:
regen_ctr = True
feature_name = "localCTR"
stat = {}

In [23]:
def ExtractLocalCTR(df, save_dir, prefix, feature_name):
    global stat
    grouped = df.groupby("query_id", as_index=False)
    count = grouped.size().values
    if not stat:
        label_sum = grouped["label"].sum()["label"].values.astype("int32")
        for _ in range(len(count)):
            item_len = count[_]
            if item_len not in stat:
                stat[item_len] = [1, label_sum[_]]
            else:
                stat[item_len][0] += 1
                stat[item_len][1] += label_sum[_]
        del label_sum
    res = [[ stat[_][1] / (_ * stat[_][0])]*_ for _ in count]
    result = []
    for _ in res:
        result.extend(_)
    del grouped, res
    gc.collect()
    df["{}_localCTR".format(prefix)] = result
    return df

In [24]:
if regen_ctr:
    ExtractFeature(train_file, feature_save_dir, train_feature_prefix, feature_name, 
    ExtractLocalCTR, names=ORI_TRAIN_NAMES,process_chunkly=False, 
    drop_first_cols=['query_title_id', 'query', 'title'], 
    drop_last_cols=['query_id', 'label'], strip=False)

----->started 'extract localCTR feature' block...
----->started 'save feature' block...
saved localCTR feature to /home/kesci/work/common/features/magic/debug_trainMagic_localCTR.h5
----->finished 'save feature' block, time used:0.6s.
----->finished 'extract localCTR feature' block, time used:3.25s.


In [33]:
# AnalysisHDF("/home/kesci/work/common/features/magic/debug_trainMagic_localCTR.h5", print_rows=10)

In [10]:
if not OFFLINE and regen_ctr:
    ExtractFeature(test_file, feature_save_dir, test_feature_prefix, feature_name, 
    ExtractLocalCTR, names=ORI_TEST_NAMES,process_chunkly=False, 
    drop_first_cols=['query_title_id', 'query', 'title'], 
    drop_last_cols=['query_id'], strip=False)

----->started 'extract localCTR feature' block...
----->started 'save feature' block...
saved localCTR feature to /home/kesci/work/common/features/magic/online_testMagic_localCTR.h5
----->finished 'save feature' block, time used:12.38s.
----->finished 'extract localCTR feature' block, time used:61.49s.


## 杂项

In [7]:
import os
print(os.getpid())

1713


In [ ]:
! kill -9 1713